In [2]:
import pandas as pd
import numpy as np
import csv
import datetime as dt
from datetime import timedelta
from datetime import datetime
import time
import matplotlib as plt
import statistics
import math
import plotly.plotly as py
from datetime import date


In [17]:
data_start = '2019-4-15'
data_end = '2019-4-30'

res = (datetime.strptime(data_end, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
adr = pd.read_csv('analytics_data_report2.csv')
df = pd.DataFrame(adr)
df.columns = df.columns.str.replace(' ', '_').str.lower()
t1 = pd.to_datetime(df['referral_received'])
t2 = pd.to_datetime(df['loan_booked'])
mask = (t1 <= res) & (t1 >= data_start)
conv = df.loc[mask]
bookcount = conv.groupby('partner')['loan_booked'].count()
leadcount = conv.groupby('partner')['referral_received'].count()
rev1 = conv.groupby('partner')['gap_gross_revenue'].mean
conversions = bookcount / leadcount
conv['difference'] = t2 - t1
conv['difference'] = conv['difference'] / dt.timedelta(days=1)
df_dates = conv[['partner', 'referral_received', 'loan_booked', 'difference']]
mean_diff = df_dates.groupby('partner')['difference'].mean()
med_diff = df_dates.groupby('partner')['difference'].median()
df_summ = pd.DataFrame({'Volume': leadcount,
                       'Booked/App': conversions,
                       'Mean Time to Fund': mean_diff,
                       'Median Time to Fund': med_diff})
result = pd.merge(df, df_summ, on='partner')
result = result[np.isfinite(result['Median Time to Fund'])]
result = result[np.isfinite(result['Mean Time to Fund'])]
start = pd.to_datetime(result['referral_received'])
result['p_fund_median'] = start + result['Median Time to Fund'].map(dt.timedelta)
result['p_fund_mean'] = start + result['Mean Time to Fund'].map(dt.timedelta)
t3 = pd.to_datetime(result['p_fund_median'])
df_summ.dropna()

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,Volume,Booked/App,Mean Time to Fund,Median Time to Fund
partner,,,,
AVIS,94,0.372340,16.891290,16.236111
CarGurus - Avis,10,0.300000,15.718287,14.188194
Chrysler Capital Lease,79,0.316456,18.851639,19.615972
Credit Karma,681,0.080764,23.770278,22.403472
Donlen,17,0.647059,15.219823,13.629167
Element,36,0.277778,31.257014,26.397917
Emkay,7,0.714286,23.409583,17.468056
Enterprise Fleet Management,77,0.766234,19.986323,17.904861
Fiserv GM Lease,74,0.364865,24.049100,22.265972


In [18]:
start_date = '2019-6-1'
end_date = '2019-6-16'

res2 = (datetime.strptime(end_date, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
pred_median = (result['p_fund_median'] <= res2) & (result['p_fund_median'] >= start_date)
new_median = result.loc[pred_median]
appcount_median = new_median.groupby('partner')['referral_received'].count()
predicted_median = appcount_median * conversions
Total_median = predicted_median.sum()
pred_mean = (result['p_fund_mean'] < res2) & (result['p_fund_mean'] >= start_date)
new_mean = result.loc[pred_mean]
appcount_mean = new_mean.groupby('partner')['referral_received'].count()
predicted_mean = appcount_mean * conversions
Total_mean = predicted_mean.sum()
booked = pd.to_datetime(result['loan_booked'])
actual = (booked <= res2) & (booked >= start_date)
actual2 = result.loc[actual]
true_actual = actual2.groupby('partner')['loan_booked'].count()
prediction_test = pd.DataFrame({'Predicted Median': round(predicted_median,0),
                           'Predicted Mean': round(predicted_mean,0),
                           'Actual': true_actual})
prediction_test.loc['Total'] = prediction_test.sum()
prediction_test.dropna()

,Predicted Median,Predicted Mean,Actual
AVIS,35.0,36.0,37.0
Chrysler Capital Lease,23.0,24.0,20.0
Credit Karma,49.0,50.0,53.0
Donlen,4.0,5.0,3.0
Element,9.0,9.0,11.0
Emkay,2.0,3.0,4.0
Enterprise Fleet Management,70.0,57.0,68.0
Fiserv GM Lease,35.0,38.0,31.0
HCA - Hyundai Kia,28.0,24.0,18.0
IFS,11.0,12.0,22.0


In [19]:
cols = list(result.columns)
cols = [cols[0]] + cols[0:]
result = result[cols]
result.to_dict()
result


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning:

DataFrame columns are not unique, some columns will be omitted.



,app_id,app_id,partner,fa_tier,fa,loan_processor,funding_clerk,title_clerk,lender,state,...,gap_gross_revenue,vsc_gross_revenue,tw_gross_revenue,total_gross_revenue,Volume,Booked/App,Mean Time to Fund,Median Time to Fund,p_fund_median,p_fund_mean
0,A19010001,A19010001,IFS Marketing Organic Lease,3,ddalton,NaN,NaN,NaN,NaN,TN,...,0.00,0,0,0.00,3,0.333333,8.725000,8.725000,2019-01-09 18:00:00,2019-01-09 18:00:00
1,A19010152,A19010152,IFS Marketing Organic Lease,3,hchang,NaN,NaN,NaN,NaN,NJ,...,0.00,0,0,0.00,3,0.333333,8.725000,8.725000,2019-01-10 13:48:00,2019-01-10 13:48:00
2,A19010204,A19010204,IFS Marketing Organic Lease,0,kberry,NaN,NaN,NaN,NaN,TX,...,0.00,0,0,0.00,3,0.333333,8.725000,8.725000,2019-01-10 19:48:00,2019-01-10 19:48:00
3,A19010734,A19010734,IFS Marketing Organic Lease,1,kstanley,ksteinmetz,NaN,bruiz,Capital One Auto Finance,TX,...,620.00,1000,0,2340.02,3,0.333333,8.725000,8.725000,2019-01-12 09:26:00,2019-01-12 09:26:00
4,A19010939,A19010939,IFS Marketing Organic Lease,1,hchang,NaN,NaN,NaN,NaN,AZ,...,0.00,0,0,0.00,3,0.333333,8.725000,8.725000,2019-01-13 04:38:00,2019-01-13 04:38:00
5,A19012018,A19012018,IFS Marketing Organic Lease,1,kberry,NaN,NaN,NaN,NaN,NY,...,0.00,0,0,0.00,3,0.333333,8.725000,8.725000,2019-01-17 01:52:00,2019-01-17 01:52:00
6,A19012071,A19012071,IFS Marketing Organic Lease,1,knguyen,NaN,NaN,NaN,NaN,MA,...,0.00,0,0,0.00,3,0.333333,8.725000,8.725000,2019-01-17 04:44:00,2019-01-17 04:44:00
7,A19012688,A19012688,IFS Marketing Organic Lease,1,tdooley,NaN,NaN,NaN,NaN,CA,...,0.00,0,0,0.00,3,0.333333,8.725000,8.725000,2019-01-18 18:02:00,2019-01-18 18:02:00
8,A19013166,A19013166,IFS Marketing Organic Lease,3,kstanley,NaN,NaN,NaN,NaN,IN,...,0.00,0,0,0.00,3,0.333333,8.725000,8.725000,2019-01-20 09:15:00,2019-01-20 09:15:00
9,A19013221,A19013221,IFS Marketing Organic Lease,1,ddalton,NaN,NaN,NaN,Consumers Credit Union,NJ,...,503.00,0,0,802.00,3,0.333333,8.725000,8.725000,2019-01-20 12:15:00,2019-01-20 12:15:00


In [20]:
df2 = df[['partner','gap_gross_revenue','vsc_gross_revenue','tw_gross_revenue','total_gross_revenue']]
df2 = df2.replace(0, np.NaN)

In [21]:
grouped = df2.groupby('partner').agg(np.mean)
grouped.dropna()


,gap_gross_revenue,vsc_gross_revenue,tw_gross_revenue,total_gross_revenue
partner,,,,
AVIS,438.167358,1097.363636,333.107143,1172.953686
Avis Online App,515.000000,1000.000000,327.000000,1137.067800
Avis Rental,484.000000,1000.000000,336.500000,1244.855846
CarGurus - Avis,501.400000,1187.571429,327.000000,1115.395875
Chrysler Capital Lease,496.229091,933.114504,330.816327,1220.403426
ConsumersAdvocate,513.333333,1000.000000,377.000000,1157.800000
Credit Karma,531.859021,975.356098,325.648780,689.992941
Credit-com,536.500000,1000.000000,427.000000,594.136364
Donlen,478.890769,1039.307692,324.571429,1094.748533
